In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('./making/covid_city.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      18 non-null     object
 1   누적확진자   18 non-null     int64 
 2   신규확진자   18 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 560.0+ bytes


In [4]:
df

,지역,누적확진자,신규확진자
0,서울,76814,569
1,경기,68671,513
2,대구,13287,95
3,인천,11406,102
4,부산,11077,78
5,경남,9683,68
6,경북,6866,64
7,충남,6639,92
8,검역,5761,23
9,대전,5463,50


# 시각화를 위해 데이터 df 만들기

In [5]:
# 검역 제거
df2 = df[df['지역']!= '검역']
df2.reset_index(drop = True)

,지역,누적확진자,신규확진자
0,서울,76814,569
1,경기,68671,513
2,대구,13287,95
3,인천,11406,102
4,부산,11077,78
5,경남,9683,68
6,경북,6866,64
7,충남,6639,92
8,대전,5463,50
9,강원,5268,26


In [6]:
# 광주 -> 광주광역시
df2 = df2.replace('광주','광주광역시')

In [7]:
df2

,지역,누적확진자,신규확진자
0,서울,76814,569
1,경기,68671,513
2,대구,13287,95
3,인천,11406,102
4,부산,11077,78
5,경남,9683,68
6,경북,6866,64
7,충남,6639,92
9,대전,5463,50
10,강원,5268,26


In [8]:
cities = df2['지역'].values

In [9]:
len(cities)

17

# 카카오 API 이용하기

### 샘플로 주소 검색 맛보기

In [10]:
import requests

In [20]:
searching_sample = '서울'
url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(searching_sample)

headers = {
    "Authorization": "KakaoAK 50b490bd64fefad5a4611cd9294fd813"     
    # 입력시 반드시  KakaoAK 뒤에 한 칸 띄우고 API키 값을 적어야 합니다. 
}

places_sample = requests.get(url, headers = headers).json()['documents']
places_sample

[{'address': {'address_name': '서울',
   'b_code': '1100000000',
   'h_code': '1100000000',
   'main_address_no': '',
   'mountain_yn': 'N',
   'region_1depth_name': '서울',
   'region_2depth_name': '',
   'region_3depth_h_name': '',
   'region_3depth_name': '',
   'sub_address_no': '',
   'x': '126.978652258309',
   'y': '37.566826004661'},
  'address_name': '서울',
  'address_type': 'REGION',
  'road_address': None,
  'x': '126.978652258309',
  'y': '37.566826004661'}]

### 실제 데이터 사용하기

In [11]:
places = []

In [12]:
for city in cities:
    searching = city
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(searching)

    headers = {
        "Authorization": "KakaoAK 50b490bd64fefad5a4611cd9294fd813"
    }

    places.extend(requests.get(url, headers = headers).json()['documents'])
    

In [13]:
len(places)

17

In [24]:
places

[{'address': {'address_name': '서울',
   'b_code': '1100000000',
   'h_code': '1100000000',
   'main_address_no': '',
   'mountain_yn': 'N',
   'region_1depth_name': '서울',
   'region_2depth_name': '',
   'region_3depth_h_name': '',
   'region_3depth_name': '',
   'sub_address_no': '',
   'x': '126.978652258309',
   'y': '37.566826004661'},
  'address_name': '서울',
  'address_type': 'REGION',
  'road_address': None,
  'x': '126.978652258309',
  'y': '37.566826004661'},
 {'address': {'address_name': '경기',
   'b_code': '4100000000',
   'h_code': '4100000000',
   'main_address_no': '',
   'mountain_yn': 'N',
   'region_1depth_name': '경기',
   'region_2depth_name': '',
   'region_3depth_h_name': '',
   'region_3depth_name': '',
   'sub_address_no': '',
   'x': '127.00892996953',
   'y': '37.2749769872425'},
  'address_name': '경기',
  'address_type': 'REGION',
  'road_address': None,
  'x': '127.00892996953',
  'y': '37.2749769872425'},
 {'address': {'address_name': '대구',
   'b_code': '2700000000

# 위도 경도 저장

In [19]:
longitude = []
latitude = []
for i in range(len(places)):
    longitude.append(places[i]['x'])
    latitude.append(places[i]['y'])

# 데이터 시각화를 위해 위도 경도 DF 만들기

In [14]:
cities

array(['서울', '경기', '대구', '인천', '부산', '경남', '경북', '충남', '대전', '강원', '충북',
       '울산', '광주광역시', '전북', '제주', '전남', '세종'], dtype=object)

In [26]:
longitude = list(map(float,longitude))
longitude

[126.978652258309,
 127.00892996953,
 128.601805491072,
 126.705258070068,
 129.075087492149,
 128.69189688916,
 128.505799255401,
 126.672776193822,
 127.38483484675,
 127.729829010354,
 127.491457326501,
 129.311467815675,
 126.851629955742,
 127.108976712011,
 126.498229141199,
 126.462788333376,
 127.289195324698]

In [27]:
latitude = list(map(float,latitude))
latitude

[37.566826004661,
 37.2749769872425,
 35.8713802646197,
 37.4560044656444,
 35.1798200522868,
 35.2377742104522,
 36.5759962255808,
 36.6588292532864,
 36.3505388992836,
 37.8853257858209,
 36.6353581959954,
 35.5394791854502,
 35.1601037626662,
 35.8201963639272,
 33.4889179032603,
 34.8160821478848,
 36.4800649113762]

In [28]:
df2['위도'] = latitude
df2['경도'] = longitude

In [29]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 0 to 17
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지역      17 non-null     object 
 1   누적확진자   17 non-null     int64  
 2   신규확진자   17 non-null     int64  
 3   위도      17 non-null     float64
 4   경도      17 non-null     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 816.0+ bytes


In [52]:
df2 = df2.reset_index(drop = True)

In [53]:
df2

,지역,누적확진자,신규확진자,위도,경도
0,서울,76814,569,37.566826,126.978652
1,경기,68671,513,37.274977,127.008930
2,대구,13287,95,35.871380,128.601805
3,인천,11406,102,37.456004,126.705258
4,부산,11077,78,35.179820,129.075087
5,경남,9683,68,35.237774,128.691897
6,경북,6866,64,36.575996,128.505799
7,충남,6639,92,36.658829,126.672776
8,대전,5463,50,36.350539,127.384835
9,강원,5268,26,37.885326,127.729829


In [45]:
len(df2)

17

In [31]:
import folium 

In [64]:
covid_map = folium.Map(location=[36, 127], zoom_start=7)

for i in range(len(df2)):
    name = df2['지역'][i]
    count = df2['신규확진자'][i]
    size = int(count) * 0.05
    long = float(df2['경도'][i])
    lat = float(df2['위도'][i])
    folium.CircleMarker((lat,long), radius = size, color='red', popup=name).add_to(covid_map)
    
covid_map